In [1]:
import numpy as np
import torch
import random
import torch.nn as nn
import pickle
import argparse
from data import DATA
import json
import os
from optimizer import OPTIM
import time
from datetime import datetime

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda


In [3]:
# Load data
data_obj = DATA(device=device)
s_time = datetime.now()
train_data, valid_data, test_data, vocab_obj = data_obj.f_load_featsent_1(
    data_name = 'ratebeer_medium',
    data_dir = '/u/pw7nc/experiments/Dataset/ratebeer/medium_500_pure',
    batch_size = 4,
    batch_size_eval = 8
)
e_time = datetime.now()
print("... load data duration ... ", e_time-s_time)

data
Loading datasets...


/u/pw7nc/anaconda3/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/u/pw7nc/anaconda3/lib/python3.7/site-packages/torchtext/data/example.py:13: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


user vocab built
feature vocab built
label vocab built
Number of batches in 1 training epoch: 1502452
Number of batches in 1 validation epoch: 75048
Number of batches in 1 testing epoch: 1409951
============== Dataset Loaded ==============
... load data duration ...  0:04:47.499523


/u/pw7nc/anaconda3/lib/python3.7/site-packages/torchtext/data/iterator.py:48: UserWarning: Iterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [4]:
len(train_data)

1502452

In [5]:
len(test_data)

1409951

In [6]:
for train_batch in train_data:
    user_batch = train_batch.user
    item_batch = train_batch.item
    feature_tensor, feature_length = train_batch.feature
    print(user_batch)
    print(item_batch)
    print(feature_tensor)
    print(feature_length)
    break

/u/pw7nc/anaconda3/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


tensor([0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0], device='cuda:0')
tensor([[  8,  28,  38],
        [ 62, 167,   1],
        [  8,  21,  36],
        [235,   1,   1]], device='cuda:0')
tensor([3, 2, 3, 1], device='cuda:0')


In [15]:
feature_length[0].item()

6

In [ ]:
for test_batch in test_data:
    user_batch = test_batch.user
    item_batch = test_batch.item
    sentence_batch = test_batch.sentence
    print(user_batch)
    print(item_batch)
    print(sentence_batch)

In [8]:
for train_batch in train_data:
    print(train_batch.sentence)
    break

tensor([97272, 54947, 97461, 97411], device='cuda:0')


/u/pw7nc/anaconda3/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [9]:
for train_batch in train_data:
    print(train_batch.user)
    print(train_batch.item)
    break

tensor([66, 66, 66, 66], device='cuda:0')
tensor([1301, 1301, 1301, 1301], device='cuda:0')


/u/pw7nc/anaconda3/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [30]:
for train_batch in train_data:
    print(train_batch.label)
    print(train_batch.label.dtype)
    break

tensor([0, 0, 0, 0], device='cuda:0')
torch.int64


/u/pw7nc/anaconda3/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [31]:
train_batch.label.float()

tensor([0., 0., 0., 0.], device='cuda:0')

In [10]:
user_voc = vocab_obj.m_uservocab
item_voc = vocab_obj.m_itemvocab
feat_voc = vocab_obj.m_textvocab
print("Number of users: {}".format(vocab_obj.user_num))
print("Number of items: {}".format(vocab_obj.item_num))
print("Number of feature vocab size: {}".format(vocab_obj.vocab_size))

Number of users: 1554
Number of items: 1396
Number of feature vocab size: 576


In [11]:
user_voc.stoi["1000"]

66

In [13]:
item_voc.stoi["1535"]

1301

In [32]:
type(feat_voc.stoi['water'])

int

In [15]:
feat_voc.itos[1]

'<pad>'

In [16]:
feat_voc.itos[0]

'<unk>'

In [18]:
feat_voc.itos[575]

'shaker'

In [20]:
type(feat_voc.stoi)

collections.defaultdict

In [25]:
for feat_word in feat_voc.stoi.keys():
    print(feat_word)
    print(type(feat_word))
    break

<unk>
<class 'str'>


In [26]:
import json
feature_emb_file = "/u/pw7nc/experiments/Dataset/ratebeer/medium_500_pure/train/feature/featureid2embedding.json"
with open(feature_emb_file, 'r') as f:
    feat_emb_origin = json.load(f)

In [28]:
type(feat_emb_origin['0'])

list

In [30]:
vocab_obj.unk_token

'<unk>'

In [31]:
vocab_obj.pad_token

'<pad>'

In [29]:
sid2sentemb_file = "/u/pw7nc/experiments/Dataset/ratebeer/medium_500_pure/train/sentence/sid2sentembed.json"
with open(sid2sentemb_file, 'r') as f:
    for line in f:
        line_data = json.loads(line)
        # sid2sentemb_voc = json.load(f)
        # print(line_data)
        print(list(line_data.keys())[0])
        print(type(list(line_data.keys())[0]))
        print(len(line_data['0']))
        break

0
<class 'str'>
768


In [19]:
import torch
tensor_1 = torch.tensor([[[1,2],[2,3]],[[1,2],[2,3]],[[2,3],[3,4]]])

In [20]:
tensor_1.shape

torch.Size([3, 2, 2])

In [21]:
tensor_1

tensor([[[1, 2],
         [2, 3]],

        [[1, 2],
         [2, 3]],

        [[2, 3],
         [3, 4]]])

In [22]:
tensor_2 = torch.sum(tensor_1, dim=1)

In [26]:
tensor_2

tensor([[3, 5],
        [3, 5],
        [5, 7]])

In [27]:
tensor_2 = tensor_2 / torch.tensor([[2],[3],[4]])

In [28]:
tensor_2

tensor([[1.5000, 2.5000],
        [1.0000, 1.6667],
        [1.2500, 1.7500]])

In [24]:
tensor_2 / torch.tensor([[2],[3],[4]])

tensor([[1.5000, 2.5000],
        [1.0000, 1.6667],
        [1.2500, 1.7500]])

In [13]:
mask_1 = torch.tensor([[1,1],[0,1],[0,1]])

In [14]:
mask_1

tensor([[1, 1],
        [0, 1],
        [0, 1]])

In [17]:
mask_1.unsqueeze(dim=-1)

torch.Size([3, 2, 1])

In [18]:
tensor_1 * (mask_1.unsqueeze(dim=-1))

tensor([[[1, 2],
         [2, 3]],

        [[0, 0],
         [2, 3]],

        [[0, 0],
         [3, 4]]])

In [7]:
mask_2 = torch.zeros(2,3)

In [8]:
mask_2

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [9]:
mask_2[0][:2] = 1

In [10]:
mask_2

tensor([[1., 1., 0.],
        [0., 0., 0.]])

In [11]:
mask_2[1][:3] = 1

In [12]:
mask_2

tensor([[1., 1., 0.],
        [1., 1., 1.]])